<a href="https://colab.research.google.com/github/ProfessorPatrickSlatraigh/CIS9557_bctw/blob/main/cis9557_GROUP_1_SuperStore_timeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CIS9557 SuperStore Time Series  
<b>Scaffolding and Data Wrangling for Group 1</b>  
<b>Using `Ship Date`, and a.) `Category`, or b.) `Region` Features</b>  
  
*example of data wrangling for time-series, forecasting*  



##0. Housekeeping  

Import the usual suspects

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
from datetime import datetime

In [4]:
from matplotlib import pyplot as plt

Load (US) sample Superstore.XLS Orders spreadsheet from CSV file on Professor Patrick's Github  


In [5]:
!curl 'https://raw.githubusercontent.com/ProfessorPatrickSlatraigh/data/main/USsampleSuperstoreOrders_May-2024.csv' -o usOrders.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2234k  100 2234k    0     0  4332k      0 --:--:-- --:--:-- --:--:-- 4339k




---



##1. Create a Date Range Scaffold  
  
Spin up a series of `timedate` values to use as scaffolding for time series analysis  


In [6]:
def timescaffold(startIncludes='01-Jan-2019', endIncludes='31-Mar-2019', periodFrequency='daily'):
    """

    Args:
      startIncludes:   the scaffold series starting date in %d-%b-$Y (dd-Mmm-YYYY) format
      endIncludes:     the scaffold series ending date in %d-%b-$Y (dd-Mmm-YYYY) format
      periodFrequency: the frequency of the scaffold series periods (daily, weekdays, businessdays, weekly, monthly, annually)
    """

    print(f'From {startIncludes} to {endIncludes} at a {periodFrequency} rate.\n')

    # set a pattern for the expected date format in strings (user entry)
    date_format = '%d-%b-%Y'  # %d for day, %b for abbreviated month, %Y for four-digit year
    # test for a missing start date parameter and prompt for a valid value
    # if startIncludes == '01-Jan-2019':
    goodStart = input(f'\nThe time series will start with {startIncludes}. Is that correct? ')
    while goodStart.lower()[0] != 'y':
        startIncludes = input('What value would you like to start with? ')
        try:
            startScaffold = datetime.strptime(startIncludes, date_format)
            goodStart='yes'
        except:
            print('Your input was not a valid date in the format "dd-Mmm-YYYY". Enter a new value.')
    startScaffold = datetime.strptime(startIncludes, date_format)
    print(f'Thank you.  The scaffold time series will begin with {startScaffold}.')
    # test for a missing end date parameter and prompt for a valid value
    # if endIncludes == '31-Mar-2019':
    goodEnd = input(f'\nThe time series will end with {endIncludes}. Is that correct? ')
    while goodEnd.lower()[0] != 'y':
        endIncludes = input('What value would you like to start with?')
        try:
            endScaffold = datetime.strptime(endIncludes, date_format)
            goodEnd = 'yes'
        except:
            print('Your input was not a valid date in the format "dd-Mmm-YYYY". Enter a new value. ')
    endScaffold = datetime.strptime(endIncludes, date_format)
    print(f'Thank you.  The scaffold time series will end with {datetime.strptime(endIncludes, date_format)}.')

    return(pd.date_range(start=startScaffold, end=endScaffold, freq='D'))



In [20]:
scaffold = timescaffold()

From 01-Jan-2019 to 31-Mar-2019 at a daily rate.


The time series will start with 01-Jan-2019. Is that correct? y
Thank you.  The scaffold time series will begin with 2019-01-01 00:00:00.

The time series will end with 31-Mar-2019. Is that correct? y
Thank you.  The scaffold time series will end with 2019-03-31 00:00:00.


In [ ]:
print(len(scaffold))

In [ ]:
print(scaffold)



---



##2. Load Profit Dataframe  
  
Create a `Dataframe` of the `ShippedDate` and `Profit` variables (columns) from the `Orders` dataset (sheet) retaining the following features:  
- `Row ID`  
- `Ship Date`  
- `Segment`  
- `State`  
- `Region`  
- `Product ID`  
- `Category`  
- `Profit` (target)  
  

Create a list of the columns to keep and the names to use for them

In [21]:
keep_columns_ls = ['Row ID', 'Ship Date', 'Category', 'Segment', 'State', 'Region', 'Product ID', 'Category', 'Profit']

In [22]:
orders_df = pd.read_csv('usOrders.csv', usecols=keep_columns_ls)

In [ ]:
orders_df.columns

In [ ]:
orders_df.describe()

Create a `datetime` value on each row from its  'Ship Date'

In [23]:
orders_df['DateShipped'] = pd.to_datetime(orders_df['Ship Date'])

In [ ]:
orders_df.head(15)

##3. Merge Timedate Scaffold and Superstore Dataframe  

Merge the time series scaffold with the dataset in the `dataframe`  

In [24]:
scaffold_df = pd.DataFrame(scaffold, columns=['Date'])

In [ ]:
scaffold_df.head(15)

In [ ]:
len(scaffold_df)

###3.a) Enrich Scaffolding for `Ship Date` and `Region`  
  
<font color=blue><i>Note: the steps in the section are to scaffold the Superstore dataset for `Ship Date` and `Region` and produce a dataset that has at least one row (record) for every combination of `Ship Date` and `Region`.  If the original dataset has more than one row for any combination of `Ship Date` and `Region` those will be aggregated so that we have only one row for every combination of desired features -- that one row will have the aggregate profit for that combination of features.</i></font>  
  
<font color=red><i>If you intend to analyze the dataset by `Ship Date` and `Category`, then use <b>Step 3.b)</b> instead of this step.</i></font>



Create a `regions_df` `dataframe` with a value for every `Region`

In [13]:
# DataFrame with the 'Region' values
regions = ['West', 'East', 'Central', 'South']
regions_df = pd.DataFrame({'Region': regions})

In [14]:
# Performing a cross join to combine every 'Date' with every 'Region'
expandedScaffold_df = scaffold_df.assign(key=1).merge(regions_df.assign(key=1), on='key').drop('key', axis=1)

In [ ]:
expandedScaffold_df.head(10)

In [ ]:
len(expandedScaffold_df)

In [15]:
training_df = pd.merge(expandedScaffold_df, orders_df, left_on=['Date', 'Region'], right_on=['DateShipped', 'Region'], how='outer').drop('DateShipped', axis=1)

Now we have an enriched dataset with all the `Date Shipped` and `Region` combinations included at least once, but some of these record may have missing values (NaN) for `Profit`.  

In [ ]:
training_df.head(20)

In [ ]:
len(training_df)

Sub-total the `Profit` data in the `training_df` dataset by `Region` and `Date`

In [16]:
# Use a specific column for more meaningful count, here using 'Value' column
summaryProfit_df = training_df.groupby(['Region', 'Date'])['Profit'].count().reset_index(name='Count')

In [ ]:
len(summaryProfit_df)

In [ ]:
# Display the result
print(summaryProfit_df)

Sub-total the `Date` instances in the `training_df` dataset by `Region` and `Date`

In [17]:
# Use a specific column for more meaningful count, here using 'Value' column
summaryDate_df = training_df.groupby(['Region', 'Date'])['Date'].count().reset_index(name='Count')

In [ ]:
len(summaryDate_df)

In [ ]:
# Display the result
print(summaryDate_df)

<center><b><font color=blue>NOW PROCEED TO STEP #4</b></font></center>  




---



###3.b) Enrich Scaffolding for `Ship Date` and `Category`  
  
<font color=blue><i>Note: the steps in the section are to scaffold the Superstore dataset for `Ship Date` and `Category` and produce a dataset that has at least one row (record) for every combination of `Ship Date` and `Category`.  If the original dataset has more than one row for any combination of `Ship Date` and `Category` those will be aggregated so that we have only one row for every combination of desired features -- that one row will have the aggregate profit for that combination of features.</i></font>  
  
<font color=red><i>If you intend to analyze the dataset by `Ship Date` and `Region`, then use <b>Step 3.a)</b> instead of this step.</i></font>



Create a `categories_df` `dataframe` with a value for every `Category`

In [25]:
# DataFrame with the 'Category' values
categories = ['Furniture', 'Office Supplies', 'Technology']
categories_df = pd.DataFrame({'Category': categories})

In [26]:
# Performing a cross join to combine every 'Date' with every 'Catgeory'
expandedScaffold_df = scaffold_df.assign(key=1).merge(categories_df.assign(key=1), on='key').drop('key', axis=1)

In [ ]:
expandedScaffold_df.head(10)

In [ ]:
len(expandedScaffold_df)

276

In [28]:
training_df = pd.merge(expandedScaffold_df, orders_df, left_on=['Date', 'Category'], right_on=['DateShipped', 'Category'], how='outer').drop('DateShipped', axis=1)

Now we have an enriched dataset with all the `Date Shipped` and `Category` combinations included at least once, but some of these record may have missing values (NaN) for `Profit`.  

In [ ]:
training_df.head(20)

In [ ]:
len(training_df)

Sub-total the `Profit` data in the `training_df` dataset by `Category` and `Date`

In [30]:
# Use a specific column for more meaningful count, here using 'Value' column
summaryProfit_df = training_df.groupby(['Category', 'Date'])['Profit'].count().reset_index(name='Count')

In [ ]:
len(summaryProfit_df)

In [ ]:
# Display the result
print(summaryProfit_df)

Sub-total the `Date` instances in the `training_df` dataset by `Category` and `Date`

In [31]:
# Use a specific column for more meaningful count, here using 'Value' column
summaryDate_df = training_df.groupby(['Category', 'Date'])['Date'].count().reset_index(name='Count')

In [ ]:
len(summaryDate_df)

In [ ]:
# Display the result
print(summaryDate_df)



---



<center><b><font color=blue>Regardless of whether sub-step 3.a or 3.b above was used, all of the following steps apply.</b></font></center>



---



##4. Fill Missing Values

We can replace missing (NaN) values for `Profit` with zeroes.  Those will typically occur on the new, scaffolder rows we created for feature combinations with `Ship Date` that did not occur in the original dataset.

In [ ]:
training_df.columns

In [ ]:
training_df.describe()

In [32]:
training_df['Profit'].fillna(0, inplace=True)



---



##5. Write CSV File

We write the resulting file as `training.csv` to the current working directory.  

In [33]:
training_df.to_csv('training.csv', sep=',', header=True, mode='w', encoding='utf-8')



---

